# Linear Regression

The goal of this notebook is to build a Linear Regression Model on the California Housing Dataset.

We will be trying to predict the MedianPrice per house in each block. Denoted as the target column. 

This price is in $100,000s

Previously we performed some initial data cleaning on this data and we will progress using that.



In [3]:
# importing pandas package
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# loading our cleaned data into  data frames
# We can use these different cleaning methods to experiment with our model and see the impact
df = pd.read_csv('data/housing_data_cleaned.csv')
df_IQR = pd.read_csv('data/housing_data_cleaned_IQR.csv') # tukey cleaned
df_STD = pd.read_csv('data/housing_data_cleaned_STD.csv') # mahalanobis

FileNotFoundError: [Errno 2] No such file or directory: 'data/housing_data_cleaned_IQR.csv'

In [ ]:
# verify our data frames looking at their info
df.info()

In [ ]:
# verify our data frames looking at their info
df_STD.info()

In [ ]:
# verify our data frames looking at their info
df_IQR.info()

### Recap of dataset characteristics

California Housing dataset
Data Set Characteristics:

* Number of Instances: 20640

* Number of Attributes: 8 numeric, predictive attributes and the target

* Attribute Information:
    - MedInc        median income in block group
    
    - HouseAge      median house age in block group
    
    - AveRooms      average number of rooms per household
    
    - AveBedrms     average number of bedrooms per household
    
    - Population    block group population
    
    - AveOccup      average number of household members
    
    - Latitude      block group latitude
    
    - Longitude     block group longitude
    
This dataset was obtained from the StatLib repository. https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

An household is a group of people residing within a home. Since the average number of rooms and bedrooms in this dataset are provided per household, these columns may take surpinsingly large values for block groups with few households and many empty houses, such as vacation resorts.

It can be downloaded/loaded using the :func:sklearn.datasets.fetch_california_housing function.

.. topic:: References

- Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
  Statistics and Probability Letters, 33 (1997) 291-297

### Note
Moving forward we will include all outliers. Once we have run through building a model once, your task will be to train another using a data set where the outliers have been removed and compare models.

In [ ]:
#viewing the head of the df
df.head()

In [ ]:
# viewing the info
df.info()

In [ ]:
# viewing the distributions
df.hist(bins = 50, figsize=(20,20));

# Linear Regression assumptions

Linear regression assumes that each target variables are linearly correlated with the target variable
- We need to investiage this

No multi-colinearity, i.e none of the features are highly correlated with each other, makes regression bad
- We need to investigate this also

Independence
- We will assume this to be the case
 - (can use the Durbin-Watson test, to asses, but we will not be covering this)

There are some other assumptions that we can test for after fitting the model 
- Homoscedasticity, by using Residual vs Predicted value plots
- Multivariate Normality, using Q-Q plots
 - (There are more formal tests for Multivariate Normality but we will not be covering those, namely <br>
   Shapiro-Wilk, Kolmogorov-Smironov, Jarque-Barre, or D’Agostino-Pearson)

#### Assesing if features are linearly correlated with the target variable

In [ ]:
# plotting scatter of all columns against eachother
pd.plotting.scatter_matrix(df,figsize = (20,20));


#### Discuss what you can observe from above, solely related to correlation with the target variable?

#### Discuss what you observe related to correlation of the features themselves?

## Lets retrieve the correlation coefficients

#### We can use the basic Pandas .corr() plot to see the <em>r</em>  values

The <b><i>r</b></i> value or Pearson Correlation Coefficient is a measure of the linear interdependence between two variables, or two sets of data.

The Pearson correlation coefficient is probably the most widely used measure for linear relationships between two normal distributed variables and thus often just called "correlation coefficient". <br>
However there other measures that are more sophisticaed such as Spearman's.<br>

Here is a straight forward explanation of how Pearson and Spearman correlation differ:<br>
https://support.minitab.com/en-us/minitab/21/help-and-how-to/statistics/basic-statistics/supporting-topics/correlation-and-covariance/a-comparison-of-the-pearson-and-spearman-correlation-methods/

Pandas has Pearsons, Spearmans and the Kendall-Tau correlations all built in.<br>
Easily accessable using the method argument

Usually, the Pearson coefficient is obtained via a Least-Squares fit and a value of 1 represents a perfect positive relation-ship, -1 a perfect negative relationship, and 0 indicates the absence of a relationship between variables.

Calculation is below for completeness:

$$r = \frac{{}\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \overline{x})^2(y_i - \overline{y})^2}}$$

In [ ]:
# Note you do not need to worry about recaling the data calling pandas.corr()
# Pearsons Correlation is a normalised measure
df.corr(method = 'pearson')

In [ ]:
df.corr(method = 'spearman')

#### We can feed this into Seaborn to add more colour to our results with a heatmap

In [ ]:
# Creating an axes on which to draw the heatmap
fig, ax =  plt.subplots(figsize = (10,10))

# Note: there is an exercise sheet on Matplot lib which you may find useful in understanding about figures and axes

# Plotting the heatmap on the axes
sns.heatmap(df.corr(method='pearson'), annot=True, linewidths=0.8, ax=ax);

- Median income has the highest correlation value 0.688 which indicates that it is linked to the target variable.  

- Longitude and latitude have negative correlation as it is inversely proportional to the target variable.

- Can you explain what this means in real terms?
  
- Comment on all other <b><i>r</b></i> values

<details>
    <summary style="color:green;font-weight:bold">Thoughts</summary>
    
Blocks with higher median incomes have higher median house prices, to be expected
    
Logtitude and lattitdue being negatively correlated suggest that further south and east have higher house prices, we will visualize this later
    
Also they are heavily correlated together
    


    
    
    
    

#### Right now we are just interested in the correlation of each feature with the target variable so we can isolate this

In [ ]:
# Calling the correlation method, but this time on just one column, rather than the whole DataFrame
df.corr(method='pearson')['Target'].sort_values(ascending = False)

## The next step is to Re-Scale the data

We will scale the features before building an OLS model on top of it. This is because SciKit-Learn will expect an intercept of 0 to build the model. By standardising the data we ensure that the line of best fit will pass through the origin as the mean of all observations will be zero.


- But before that we need to split the data

- If we rescale over the entire data, i.e before we split the data, we will induce what is referred to as <b><i>Data Leakage</b></i>.

This refers to information from our test set (which represents the **unseen**) having an impact on our model building process, thus not truly being unseen.

## Importance of the Train Test split and how it is done

It is crucial that we test our model on data that it has not seen before so we can gauge how well it might perform in the real world on real unseen data.

An important part of splitting the data is to ensure there is a broad range of of the target variable in each split.

If we didn't ensure this, it is possible that we could train our model on the lowest target values and then test it on the highest. This would severly bias our model and the underperformance would be a Reducible Error

Scikit-learn's `train_test_split` ensures that we have a similar distribution of our target variable across both the training and testing sets. 

We will split the data and observe:

In [ ]:
df

In [ ]:
df['Target']

In [ ]:
df[['MedInc', 
    'HouseAge', 
    'AveRooms', 
    'AveBedrms', 
    'Population', 
    'AveOccup',
    'Latitude', 
    'Longitude']]

In [ ]:
# Importing package
from sklearn.model_selection import train_test_split

X = df[['MedInc', 
        'HouseAge', 
        'AveRooms', 
        'AveBedrms', 
        'Population', 
        'AveOccup',
        'Latitude', 
        'Longitude']]

y = df['Target']

#Splitting the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state =42)

In [ ]:
# visualising the distribution of the target in the training set
y_train.hist()

In [ ]:
# visualising the distribution of the target in the training set
y_test.hist()

We have near identical distributions across both sets.

This ensures that both the training and testing contain a representitive range of the target variable.

## Next 
We fit and apply scaling to our Training data, then apply this scaling algorithm to the Test data.

We will be applying StandardScalar() to the data, this is transforming the data to thier Z scores, ensuring they have a mean of 0 and a Standard Deviation of 1.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

$$z= \frac{x - \mu}{\sigma}$$
<br>

<center>$z$ is the rescaled value </center>
<center>$x$ is the original value </center>
<center>$\mu$ is the mean </center>
<center>$\sigma$ is the standard deviation</center>

In this case we are manually scaling the target variable

The downside of this is we must reverse this transform later in order to get the error metrics

There are more sophisticaed methods to do this in Sci-Kit learn using Pipelines and Transformed Target Regressor

But we will progress by hand to understand what is happening

In [ ]:
name = "Thom" #variable
myfunc = print #function
StandardScaler # class -> object (i.e. a piece of data). StandardScaler() == create a standard scaler object

In [ ]:
# importing the package
from sklearn.preprocessing import StandardScaler

# Calling the method twice, once for the features and once for the target
scaler =  StandardScaler()
scaler_target = StandardScaler()

In [ ]:
# Sci-Kit learn returns an array by default, we will be putting the data into another dataframe, 
# Extracting the column names for this purpose
cols = list(X_train.columns)
cols

In [ ]:
X_train

In [ ]:
print(scaler.fit_transform(X_train))

In [ ]:
# Fitting the scaler on the training features and applying it to both the training and test data
# Note how the Scaler is only fit to the training set, but then applied to both
X_train[cols] = scaler.fit_transform(X_train)
X_test[cols] = scaler.transform(X_test)


In [ ]:
X_train

In [ ]:

# Fitting the scaler on the training target and applying it to both the training and test data
y_train = scaler_target.fit_transform(y_train.values.reshape(-1, 1))
y_test = scaler_target.transform(y_test.values.reshape(-1, 1))


In [ ]:
y_train#.reshape(-1)

In [ ]:
# Viewing the transformed training data
X_train.describe()

In [ ]:
# Viewing the transformed test data
X_test.describe()

In [ ]:
# observing for the target 
print(y_train.std(), y_train.mean())
print(y_test.std(), y_test.mean())

## Lets build our first model

We will begin with a simple linear model with one input variable or feature

In [ ]:
# Viewing columns that can be selected as features for our model
X_train.columns

In [ ]:
# Setting the features that we will use
cols = ['MedInc']

More info on the model and parameters available<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# import the necessary package
from sklearn.linear_model import LinearRegression

# call the method
regressor = LinearRegression(fit_intercept = False)

# Fit the model <- THIS IS ML
regressor.fit(X_train[cols], y_train)

# Carry out predictions on our test set
y_pred = regressor.predict(X_test[cols])

In [ ]:
# Viewing the predictions
y_pred

# What just happened?


Scikit-Learn built a simple linear regression on the standardised training data using OLS, we then used the test data to make predictions, hence testing the model on data it had not seen before.

Below we will display the training data and the model created.

<center>Simple linear regression</center>

$$ y = mx+c $$ 
    
<br>
<center>$y =$ target - MedPrice</center>
<center>$x =$ MedInc</center>
<center>$m =$ Coefficient (gradient) from the model</center>
<center>$c =$Intercept</center> 
<center>which will be (0,0) as we standardized all our scores, i.e both X and Y have a mean of zero</center>

In [ ]:
# accessing model coefficient, the gradient
regressor.coef_

In [ ]:
# accessing intercept
regressor.intercept_

In [ ]:
# accessing feature used
regressor.feature_names_in_

# Plotting Model against training data

In [ ]:
plt.figure(figsize=(15, 8))
# Plotting MedInc against the Target
plt.scatter(X_train[cols], y_train, s=0.1)


# Selecting x values to input into the model
# Started at the min value of MedInc that we have, 
# the upper limit was found by experimenitng so as not to exceed the y value
# in this case the linear model is very poor at predicting anything over a standardised x value of over 3
x = np.linspace(X_train[cols].min(), 3.6, 100)

# Using the gradient and intercept from the model to plot 
# The intercept is zero but it is added for completeness
# y = m              *x+ c
y = (regressor.coef_)*x+(regressor.intercept_)

# # Plotting the linear model
plt.plot(x, y, '-r')

# # Adding labels to each axis
plt.xlabel('MedInc Z_scores')
plt.ylabel('Target Z_scores')

# calling the plot
plt.show()


# Plotting model against Test Data

In [ ]:
# Same as above
# notice the difference in the scatter plot
plt.figure(figsize=(15, 8))
plt.scatter(X_test[cols], y_test, s=0.1)
x = np.linspace(X_train[cols].min(),3.6,100)
y = (regressor.coef_)*x+(regressor.intercept_)

plt.plot(x, y, '-r', label='Model')
plt.xlabel('MedInc Z_scores')
plt.ylabel('Target Z_scores')
plt.show()

#### Assesing the model

As we have scaled the target variable, it makes sense to descale it to measure the RMSE, Root Mean Square Error of the model

If we leave it scaled the units won't make sense, however after we convert it back and find the RMSE then the error is in the same units as the data was in the first place

In [ ]:
# first we need to reverse the transformation so we have meanginful results
# calling inverse_transform on our scaler
y_true = scaler_target.inverse_transform(y_test) # <- observed y values
y_pred_inverse = scaler_target.inverse_transform(y_pred) # <- predicted y

Now to measure the errors

These errors are the differences from our predicted target values to the actual target values of the test set

In [ ]:
# Import the necessary packages
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculating the errors
mae = mean_absolute_error(y_true, y_pred_inverse)
mse = mean_squared_error(y_true, y_pred_inverse)
rmse = np.sqrt(mse)

# Calculting the R^2
r2 = r2_score(y_true, y_pred_inverse)

In [ ]:
# Viewing these results
print('MAE = ', mae)
print('MSE = ', mse)
print('RMSE = ', rmse)
print('R^2 = ', r2)

In [ ]:
# We can view the coefficients of the features
regressor.coef_

In [ ]:
# The feature names included
regressor.feature_names_in_

These results are in line with what we found before using the correlation matrix<br>
The $r^2$ value is roughly the square of the correlation coefficitent of 0.688 that we originally got.



# Multi-Linear Regression

We will now add in another variable and see its impact on the errors

In [ ]:
X_train.columns

In [ ]:
cols =['MedInc','AveRooms']

In [ ]:
# call the method
regressor = LinearRegression( fit_intercept = False)

# Fit the model
regressor.fit(X_train[cols],y_train)

# Carry out predictions on our test set
y_pred = regressor.predict(X_test[cols])

# first we need to reverse the transformation so we have meanginful results
y_true = scaler_target.inverse_transform(y_test)
y_pred = scaler_target.inverse_transform(y_pred)

# Calculating the errors
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

# Calculting the R^2
r2 = r2_score(y_true, y_pred)

# Viewing these results
print('MAE = ', mae.round(4))
print('MSE = ', mse.round(4))
print('RMSE = ', rmse.round(4))
print('R^2 = ', r2.round(4))

In [ ]:
regressor.score(X_test[cols],y_test)

By adding in AveRooms, we made minimal difference to the model

What do the above figures represent?<br>


MAE =  0.6236  <br>
A mean absolute error of 0.624 means that on average we are $60,000 off predicting the median price of a house in a block.

MSE =  0.7096<br>
The mean square error is what the model used to build itself, it has no physical meaning, techincally it is in (dollars/100,000)^2

RMSE =  0.8424<br>
A root mean square error of 0.84 tell us that we are $84,000 on average off when predicting.<br> This measure as opposed to MAE is affected more by predictions that were further off.<br> Our <i>worst</i> case scenarios have inflated this as opposed to the MAE.

R^2 =  0.4778<br>
The coefficient of determination  is defined as
<centre><b><em>1-(u/v)</em></b></centre>, where<br> <b><em>u</em></b> is the residual sum of squares ((y_true - y_pred)^ 2).sum() and<br> <b><em>v</em></b> is the total sum of squares ((y_true - y_true.mean()) ^ 2).sum(). <br>The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse)


## Trying different combinations of features

By using some built in methods in Python we can experiment with different combinations of features and asses the quality of the model.<br>
Below I will try every possible combination of 2 features and record the RMSE for each.

When finished, we will look at the top and bottom 5 combinations

The method used will be <em>combinations</em> from the built in <em>itertools</em> package <br> You can find out more here<br>https://www.geeksforgeeks.org/itertools-combinations-module-python-print-possible-combinations/

In [ ]:
# Quick demo
# importing required packages
from itertools import combinations

# Combinations takes two arguments
# The first is  some kind if iterable
# The second, how many items we can have in our combinations
# print(list(combinations('word',2)))


# experiment with the code below
possibilities = list(combinations(['A','B','C','D'],2))
possibilities

In [ ]:
# Importing required packages
from itertools import combinations

# Creating a list of all columns
all_features = list(X_train.columns)

# List of all possible combinations of 2 columns
sub_set = list(combinations(all_features,2))

# creating an empty dict to store results in
results = {}

# looping over each item in the seb set, which is a tuple containing a pair of column names
for features in sub_set:
    
    # creating a list from this tuple which will make it easier to select the columns from pandas
    # when selecting columns in pandas you need to supply it with a list
    cols = list(features)

    # call the method
    regressor = LinearRegression(fit_intercept = False)

    # Fit the model
    regressor.fit(X_train[cols],y_train)

    # Carry out predictions on our test set
    y_pred = regressor.predict(X_test[cols])

    # first we need to reverse the transformation so we have meanginful results
    y_true = scaler_target.inverse_transform(y_test)
    y_pred = scaler_target.inverse_transform(y_pred)

    # Calculating the errors
    # Only RMSE is needed right now, but the rest are here for completeness
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # Calculting the R^2
    r2 = r2_score(y_true, y_pred)
    
    # Creating an entry in the results dictionary
    # The ket is the column names
    # The value is the RMSE
    results[str(cols)] = rmse

    
# After we have looped through all combinations and saved thee values to the dict
# A dataframe is created from the dict
df_results = pd.DataFrame.from_dict(results, orient = 'index')

# The dataframe uses the keys as the index, i dont want this so i reset the index
df_results.reset_index(inplace=True)

# Rename the columns to reflect their content
df_results.columns = ['Features','RMSE']

# Sort by RMSE
df_results.sort_values(by='RMSE', inplace=True);

In [ ]:
df_results.head()

In [ ]:
df_results.tail()

#### It seems reasonable that if we chose a model with 2 features, we should select MedInc and HouseAge
- The have correlation with the target variable
- Adding HouseAge, that has low correltion with the target variable on its own, does seem to help the model
- There is no multicolinearity between them

## Task
### Review the notebook
### Fit a model using all features
### Then \\/\\/\\/\\/
### Iterate through other combinations of features and record their RMSE in a Data Frame as above

#### <i>Extension, see if you can loop through all possible combinations</i>

<details>
    <summary style="color:green;font-weight:bold">Solution</summary>

    # defining list of al columns
    all_features = list(X_train.columns)
    
    # creating a dictionary to hold results
    results = {}

    # Looping through number of possible combinations of features
    for number_of_features in range(1,len(X_train.columns)+1):

        # create all combinations of this number
        sub_set = list(combinations(all_features,number_of_features))

        # looping through all combinations
        for features in sub_set:
    
            # defining the list of features from this combination
            cols = list(features)

            # call the method
            regressor = LinearRegression( fit_intercept = False)

            # Fit the model
            regressor.fit(X_train[cols],y_train)

            # Carry out predictions on our test set
            y_pred = regressor.predict(X_test[cols])

            # first we need to reverse the transformation so we have meanginful results
            y_true = scaler_target.inverse_transform(y_test)
            y_pred = scaler_target.inverse_transform(y_pred)

            # Calculating the errors
            mae = mean_absolute_error(y_true, y_pred)
            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)

            # Calculting the R^2
            r2 = r2_score(y_true, y_pred)

            results[str(cols)] = rmse

    df_results = pd.DataFrame.from_dict(results, orient = 'index')
    df_results.reset_index(inplace=True)
    df_results.columns = ['Features','RMSE']
    df_results.sort_values(by='RMSE', inplace=True);

# Summary

The lowest RMSE we get is with 
'MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'AveOccup', 'Latitude', 'Longitude'<br>
adding 'population' to this lowers the error


We may get further improvements if we combined Longtitude and Lattitude somehow as they are heavily correlated, and eliminating multi-colinearity should help our models performance.

This will be left for you to think about and take away with you.


Below I have plotted Longtitude against Lattitude, colured by the target.

It is clear that properties closer to the sea are more expensive.


Could we combine these coordinates in a way to classify if blocks were near the sea or not?





In [ ]:
# Plotting Longitude against Latitude
# coloured by the target variable and setting the colour map
df.plot.scatter(x='Longitude' , 
                y = 'Latitude', 
                c='Target',
                s = df['Population']/100,
                cmap = 'coolwarm', 
                figsize = (20,20))


# I experimented with some lines to see if I could draw a boundary that may help with classifying the locations
# This is quite a crude method, but it would be a starting point to change the measuremnts of Long and Lat
# To close to or far from the sea
# A better way would be to verify with some geographical data
point1 = [df['Longitude'].min(), df['Latitude'].max()-1.5]
point2 = [df['Longitude'].max()-2, df['Latitude'].min()]
x_values = [point1[0], point2[0]]
y_values = [point1[1], point2[1]]


plt.plot(x_values, y_values, 'r', linestyle="--")
plt.show()

###  Extension Task

You can try rebuilding the model with the data sets with outliers removed too see the difference in the models performances

Or you could decide on which are outliers yourself and build the model on that data.

Write a brief summary of what impact the removal of outliers has had on the model. Would the model be better or worse at predicting the median price of certain types of houses?

Has it's ability to predict one kind of house increased? Has another decreased?

In [ ]:
df_STD.head()

In [ ]:
df_IQR.head()

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
SGDRegressor()

## Gradient Descent Regression from Scratch

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

rng = np.random.RandomState(42)
x = np.linspace(-3, 3, 100)
y = np.sin(4 * x) + x + rng.uniform(size=len(x))

def residuals(y, yhat):
    return y - yhat

def mean_square_error(y, yhat):
    assert y.shape == yhat.shape
    return np.sum(residuals(y, yhat) ** 2) / y.size

def d_w0(y, yhat):
    return (-2 * np.sum(residuals(y, yhat))) / y.size

def d_w1(y, yhat, x):
    return (-2 * np.sum(x * residuals(y, yhat))) / y.size

#Hyperparameter -> number of iterations/loops/models attempted before final generated
epochs = 1000

#
w1 = 0
w0 = 0

# Keep track of mse to see if improving or not
error = []

#Hyperparemeter: Learning rate -> term which scales the change in theta_0/w_0, theta_1, w_1
alpha = 0.01

for _ in range(epochs):
    yhat = w1 * x + w0
    #print(f'w0: {w0}, d_w0: {d_w0(y, yhat)}, jump: {alpha * d_w0(y, yhat)}')
    w0 -= alpha * d_w0(y, yhat)
    w1 -= alpha * d_w1(y, yhat, x)
    
    error.append(mean_square_error(y, yhat))

plt.figure(figsize=(15,8))
sns.scatterplot(x=x, y=y, label= 'data')
sns.lineplot(x=x, y=yhat, color ='red', label = 'model')
    
#plt.plot([np.min(x), np.max(x)], [np.min(w1*x + w0), np.max(w1*x + w0)])
#plt.scatter(x, y)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(range(epochs), error)

In [ ]:
mean_square_error(y, yhat)